In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import transformers
from transformers import pipeline, AutoTokenizer, DataCollatorForLanguageModeling, Gemma3ForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig,AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model
from sentence_transformers import SentenceTransformer
# trl: Transformer Reinforcement Learning library

# from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
# from trl import create_reference_model
# from trl.core import LengthSampler

import torch
import evaluate
import time

from tqdm import tqdm
tqdm.pandas()

In [ ]:
model_id = "google/gemma-2b-it"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


# Inference pipeline
chat = pipeline("text-generation", model=model, tokenizer=tokenizer)

lora_config = LoraConfig(
    r=64, # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM #GEMMA
)

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

peft_model = get_peft_model(model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

peft_model_base = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16, device_map = "cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_id)
peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       "./peft-snowboard-llm-checkpoint-local", 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)
peft_model.eval()
chat_ft = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)

In [ ]:
prompt = "hi"
response = chat_ft(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)[0]['generated_text']
response = response.replace(prompt, "").strip()
print(response)

In [ ]:
def format_messages(sample):
    messages = sample["messages"]
    prompt = ""
    for m in messages:
        role = m["role"]
        content = m["content"]
        if role == "system":
            prompt += f"<|system|>\n{content}\n"
        elif role == "user":
            prompt += f"<|user|>\n{content}\n"
        elif role == "assistant":
            prompt += f"<|assistant|>\n{content}\n"
    return {"text": prompt.strip()}
def tokenize(sample):
    result = tokenizer(
        sample["text"],
        truncation=True,
        padding="max_length",
        max_length=1000,
    )
    result["labels"] = result["input_ids"].copy()
    return result

dataset = load_dataset("json", data_files = ["snowboard_data.json", "binding_data.json"])
formatted_data = training_data.map(format_messages)
formatted_data = formatted_data.map(tokenize, remove_columns=formatted_data.column_names)

torch.cuda.empty_cache()
output_dir = f'./peft-snowboard-info-training-{str(int(time.time()))}'
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=False
)
peft_training_args = TrainingArguments(
    output_dir=output_dir,  
    gradient_accumulation_steps=16, 
    learning_rate=3e-4,            
    weight_decay=0.01,             
    warmup_ratio=0.1, 
    fp16=True,
    logging_steps=50,
    save_steps = 500,
    remove_unused_columns=False,
    save_total_limit=2 
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=formatted_data,
    data_collator=data_collator,
)

peft_trainer.train()

peft_model_path="./peft-snowboard-llm-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)